In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
import os
import warnings
warnings.filterwarnings('ignore')
os.environ["PYTHONWARNINGS"] = "ignore"

In [3]:
cur_folder_name = os.getcwd().split('/')[-1]
if cur_folder_name != "fairness-variance":
    os.chdir("../../../..")

print('Current location: ', os.getcwd())

Current location:  /Users/denys_herasymuk/UCU/4course_2term/Bachelor_Thesis/Code/fairness-variance


In [4]:
import copy
from virny.datasets import StudentPerformancePortugueseDataset
from virny.preprocessing.basic_preprocessing import preprocess_dataset
from aif360.metrics import BinaryLabelDatasetMetric
from aif360.datasets import BinaryLabelDataset

from source.preprocessing import get_simple_preprocessor, apply_lfr
from configs.constants import EXPERIMENT_SEEDS
from IPython.display import Markdown, display

pip install 'aif360[LawSchoolGPA]'
pip install 'aif360[AdversarialDebiasing]'
pip install 'aif360[AdversarialDebiasing]'


In [5]:
exp_iter_num = 1
experiment_seed = EXPERIMENT_SEEDS[exp_iter_num - 1]
test_set_fraction = 0.25
sensitive_attr_for_intervention = 'sex_binary'
privileged_groups = [{sensitive_attr_for_intervention: 1}]
unprivileged_groups = [{sensitive_attr_for_intervention: 0}]
intervention_options = {'k': 10, 'Ax': 0.1, 'Ay': 1.0, 'Az': 2.0}

data_loader = StudentPerformancePortugueseDataset()
data_loader.X_data.head()

,age,Medu,Fedu,traveltime,studytime,failures,famrel,freetime,goout,Dalc,...,reason,guardian,schoolsup,famsup,paid,activities,nursery,higher,internet,romantic
0,18,4,4,2,2,0,4,3,4,1,...,course,mother,yes,no,no,no,yes,yes,no,no
1,17,1,1,1,2,0,5,3,3,1,...,course,father,no,yes,no,no,no,yes,yes,no
2,15,1,1,1,2,0,4,3,2,2,...,other,mother,yes,no,no,no,yes,yes,yes,no
3,15,4,2,1,3,0,3,2,2,1,...,home,mother,no,yes,no,yes,yes,yes,yes,yes
4,16,3,3,1,2,0,4,3,2,1,...,home,father,no,yes,no,no,yes,yes,no,no


## Preprocessing

In [6]:
init_data_loader = copy.deepcopy(data_loader)
data_loader.categorical_columns = [col for col in data_loader.categorical_columns if col != 'sex']
data_loader.X_data[sensitive_attr_for_intervention] = data_loader.X_data['sex'].apply(lambda x: 1 if x == 'M' else 0)
data_loader.full_df = data_loader.full_df.drop(['sex'], axis=1)
data_loader.X_data = data_loader.X_data.drop(['sex'], axis=1)

# Preprocess the dataset using the defined preprocessor
column_transformer = get_simple_preprocessor(data_loader)
base_flow_dataset = preprocess_dataset(data_loader, column_transformer, test_set_fraction, experiment_seed)
base_flow_dataset.init_features_df = init_data_loader.full_df.drop(init_data_loader.target, axis=1, errors='ignore')

# Align indexes of base_flow_dataset with data_loader for sensitive_attr_for_intervention column
base_flow_dataset.X_train_val[sensitive_attr_for_intervention] =\
    data_loader.X_data.loc[base_flow_dataset.X_train_val.index, sensitive_attr_for_intervention]
base_flow_dataset.X_test[sensitive_attr_for_intervention] =\
    data_loader.X_data.loc[base_flow_dataset.X_test.index, sensitive_attr_for_intervention]

In [7]:
# Fair preprocessing
processed_base_flow_dataset = apply_lfr(base_flow_dataset,
                                        intervention_options=intervention_options,
                                        sensitive_attribute=sensitive_attr_for_intervention)

step: 0, loss: 0.6524521260855013, L_x: 1.0262899921027302,  L_y: 0.5340197646436122,  L_z: 0.007901681115808017
step: 250, loss: 0.6524521365636683, L_x: 1.0262900337852208,  L_y: 0.5340197762293759,  L_z: 0.007901678477885133
step: 500, loss: 0.6524521472368565, L_x: 1.0262900335727598,  L_y: 0.5340197781464459,  L_z: 0.007901682866567342
RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =          580     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  6.52452D-01    |proj g|=  1.18577D-01
step: 750, loss: 0.5854488897706073, L_x: 1.0261311338368373,  L_y: 0.4673266657401324,  L_z: 0.007754555323395568
step: 1000, loss: 0.5854488711116499, L_x: 1.0261311106372397,  L_y: 0.4673266487427807,  L_z: 0.007754555652572606

At iterate    1    f=  5.85449D-01    |proj g|=  8.41969D-02
step: 1250, loss: 0.5416969434487003, L_x: 1.024824662363953,  L_y: 0.4248215196233109,  L_z: 0.00719647879449706
step: 1500, loss: 0.5

In [8]:
base_flow_dataset.X_train_val.head()

,cat__school_GP,cat__school_MS,cat__address_R,cat__address_U,cat__famsize_GT3,cat__famsize_LE3,cat__Pstatus_A,cat__Pstatus_T,cat__Mjob_at_home,cat__Mjob_health,...,num__famrel,num__freetime,num__goout,num__Dalc,num__Walc,num__health,num__absences,num__G1,num__G2,sex_binary
230,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,...,-1.018120,0.792490,1.560776,0.535384,1.355964,-1.698518,2.921358,-0.147957,-0.880705,0
605,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,...,1.142606,1.729596,1.560776,0.535384,0.577926,-1.010618,-0.789879,-2.305170,-3.962470,0
471,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,...,0.062243,0.792490,0.702702,0.535384,0.577926,1.053081,-0.325974,0.211579,-0.195869,0
532,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,...,0.062243,-1.081720,-1.013445,-0.548768,1.355964,1.053081,-0.325974,-0.867028,-1.565542,1
352,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,...,0.062243,1.729596,-0.155371,-0.548768,-0.200113,1.053081,-0.789879,-0.507493,-0.538287,1


In [9]:
processed_base_flow_dataset.X_train_val.head()

,cat__school_GP,cat__school_MS,cat__address_R,cat__address_U,cat__famsize_GT3,cat__famsize_LE3,cat__Pstatus_A,cat__Pstatus_T,cat__Mjob_at_home,cat__Mjob_health,...,num__failures,num__famrel,num__freetime,num__goout,num__Dalc,num__Walc,num__health,num__absences,num__G1,num__G2
230,0.660149,0.335427,0.454704,0.562415,0.317522,0.529496,0.455574,0.473793,0.290289,0.249689,...,0.274723,0.349106,0.411284,0.468659,0.302528,0.517579,0.421129,0.403504,0.655247,0.464496
605,0.140477,0.794199,0.605305,0.079378,0.522725,0.128182,0.195926,0.508174,0.718652,0.608983,...,1.541725,0.278814,0.799233,0.667141,0.642846,0.710076,0.173493,0.683744,-1.323426,-1.885186
471,0.656988,0.384427,0.440633,0.547319,0.323522,0.483514,0.512905,0.480286,0.352521,0.249027,...,0.265181,0.345374,0.453999,0.473091,0.314183,0.497842,0.413131,0.364751,0.581734,0.499019
532,0.642330,0.358402,0.456598,0.533277,0.306980,0.479448,0.447185,0.487615,0.306483,0.268783,...,0.337188,0.327243,0.426231,0.467118,0.330684,0.519982,0.408664,0.399476,0.522252,0.319202
352,0.725124,0.318773,0.441993,0.572088,0.306664,0.501160,0.518506,0.497770,0.285413,0.225306,...,0.211779,0.343095,0.394377,0.446404,0.319131,0.496165,0.453764,0.338266,0.705939,0.598762


## Check if the intervention was applied correctly

In [10]:
processed_base_flow_dataset.X_train_val[sensitive_attr_for_intervention] = \
    data_loader.X_data.loc[base_flow_dataset.X_train_val.index, sensitive_attr_for_intervention]
processed_base_flow_dataset.X_test[sensitive_attr_for_intervention] = \
    data_loader.X_data.loc[base_flow_dataset.X_test.index, sensitive_attr_for_intervention]

# Original
original_train_df = base_flow_dataset.X_train_val
original_train_df[base_flow_dataset.target] = base_flow_dataset.y_train_val
original_test_df = base_flow_dataset.X_test
original_test_df[base_flow_dataset.target] = base_flow_dataset.y_test

dataset_orig_train = BinaryLabelDataset(df=original_train_df,
                                        label_names=[base_flow_dataset.target],
                                        protected_attribute_names=[sensitive_attr_for_intervention],
                                        favorable_label=1,
                                        unfavorable_label=0)
dataset_orig_test = BinaryLabelDataset(df=original_test_df,
                                       label_names=[base_flow_dataset.target],
                                       protected_attribute_names=[sensitive_attr_for_intervention],
                                       favorable_label=1,
                                       unfavorable_label=0)

# Transformed
transf_train_df = processed_base_flow_dataset.X_train_val
transf_train_df[processed_base_flow_dataset.target] = processed_base_flow_dataset.y_train_val
transf_test_df = processed_base_flow_dataset.X_test
transf_test_df[processed_base_flow_dataset.target] = processed_base_flow_dataset.y_test

dataset_transf_train = BinaryLabelDataset(df=transf_train_df,
                                          label_names=[processed_base_flow_dataset.target],
                                          protected_attribute_names=[sensitive_attr_for_intervention],
                                          favorable_label=1,
                                          unfavorable_label=0)
dataset_transf_test = BinaryLabelDataset(df=transf_test_df,
                                         label_names=[processed_base_flow_dataset.target],
                                         protected_attribute_names=[sensitive_attr_for_intervention],
                                         favorable_label=1,
                                         unfavorable_label=0)

In [15]:
metric_orig_train = BinaryLabelDatasetMetric(dataset_orig_train,
                                             unprivileged_groups=unprivileged_groups,
                                             privileged_groups=privileged_groups)
metric_transf_train = BinaryLabelDatasetMetric(dataset_transf_train,
                                               unprivileged_groups=unprivileged_groups,
                                               privileged_groups=privileged_groups)
display(Markdown("#### Training dataset"))
print("Transformed: Statistical Parity Difference between unprivileged and privileged groups = %f" \
      % metric_transf_train.statistical_parity_difference())
print("Original: Statistical Parity Difference between unprivileged and privileged groups = %f" \
      % metric_orig_train.statistical_parity_difference())

metric_orig_test = BinaryLabelDatasetMetric(dataset_orig_test,
                                            unprivileged_groups=unprivileged_groups,
                                            privileged_groups=privileged_groups)
metric_transf_test = BinaryLabelDatasetMetric(dataset_transf_test,
                                              unprivileged_groups=unprivileged_groups,
                                              privileged_groups=privileged_groups)
display(Markdown("#### Test dataset"))
print("Transformed: Statistical Parity Difference between unprivileged and privileged groups = %f" \
      % metric_transf_test.statistical_parity_difference())
print("Original: Statistical Parity Difference between unprivileged and privileged groups = %f" \
      % metric_orig_test.statistical_parity_difference())

#### Training dataset

Transformed: statistical_Parity Difference between unprivileged and privileged groups = 0.042717
Original: Difference in mean outcomes between unprivileged and privileged groups = 0.042717


#### Test dataset

Transformed: Difference in mean outcomes between unprivileged and privileged groups = 0.100370
Original: Difference in mean outcomes between unprivileged and privileged groups = 0.100370


In [16]:
display(Markdown("#### Individual fairness metrics"))
print("Consistency of labels in transformed training dataset= %f" %metric_transf_train.consistency())
print("Consistency of labels in original training dataset= %f" %metric_orig_train.consistency())
print("Consistency of labels in transformed test dataset= %f" %metric_transf_test.consistency())
print("Consistency of labels in original test dataset= %f" %metric_orig_test.consistency())

#### Individual fairness metrics

Consistency of labels in transformed training dataset= 0.894239
Consistency of labels in original training dataset= 0.888889
Consistency of labels in transformed test dataset= 0.868712
Consistency of labels in original test dataset= 0.857669


In [17]:
def check_algorithm_success():
    """Transformed dataset consistency should be greater than original dataset."""
    assert metric_transf_test.consistency() > metric_orig_test.consistency(), \
        "Transformed dataset consistency should be greater than original dataset."

    print('Success!')

check_algorithm_success()    

Success!
